# Dungeons and Data

This post is a first in a series of posts of applying machine learning to dungeons and dragons. This blog post details my findings with applying reinforcement learning algorithms in a simple [Dungeons and Dragons combat](https://www.youtube.com/watch?v=7tnrATiclg4) scenario. Future blog posts will include:

 * Applying RL algorithms to more complicated combat scenarios
 * Applying NLP to the story telling aspect of D&D 
 
In the spirit of full disclosure, this blog post was mainly made because I could not definitively figure out why my DQN agent failed to learn a reasonable strategy (details below).

## Combat Scenario Description

This section discusses the environment in which combat takes place. 

In order to first allow for learning in a simple environment, the combat takes place in a 50ft x 50ft room and only involves two combatants:

1. Leotris:
    * Hit points: 25
    * Armor class: 16
    * Speed: 30ft
    * Shoot arrow attack: 
        * 60 ft range
        * Hit bonus: +5
        * Damage dice: 1d12
        * Damage bonus: +3
    * Initial coordinates: (5, 5)


2. Strahd:
    * Hit points: 200
    * Armor class: 16
    * Speed: 30ft
    * Vampire bite attack:
        * 5 ft range
        * Hit bonus: +10
        * Damage dice: 3d12
        * Damage bonus: +10
    * Initial coordinates: (5, 10)
    
Each combatant, along with the attacks listed above, is allowed the following actions:
    * MoveLeft: move left 5ft
    * MoveRight: move right 5ft
    * MoveDown: move down 5ft
    * MoveUp: move up 5ft
    * EndTurn: ends turn
    
Additionally, a "time limit" of 1500 actions was implemented. In other words, agents were permitted to take a maximum of 1500 actions within a single combat encounter. 

## Goals and Learning Environment

For this experiment, `Leotris` was assigned one of the RL algorithms described below while `Strahd` was assigned a `Random` strategy in which actions were chosen at random.

The scenario was purposefully set up such that Strahd had an obvious advantage, but Leotris may be able learn to keep his distance from Strahd and use his `ShootArrow` attack in order to win. 

The following are learning goals I envisioned for this project:

* `Leotris` out performs strategy of taking random actions.
* `Leotris` learns to dealt damage. The challenge with this goal is that the agent must learn that it cannot just repeatedly take the `ShootArrow` action within the same turn. Instead, due to D&D combat rules, the agent must take the `EndTurn` action in between each `ShootArrow` usage if damage is to be done.
* `Leotris` learns to avoid damage. `Leotris` can only take damage from `Strahd` if they are within 5 ft of each other. 

In order to accomplish the above goals, the agent's "state" consists of the following:

1. Current hit points
2. Enemy hit points
3. Current x-coordinate
4. Current y-coordinate
5. Enemy x-coordinate
6. Enemy y-coordinate
7. Number of attacks used this turn
8. Remaining movement available this turn
9. Time limit remaining (as a percentage of the time limit)

A reward of `5` was given if `Leotris` was the winner. Otherwise, a reward of `0` was given if `Leotris` had taken enough actions to reach the "time limit" (1500 actions) or his hit points were reduced to zero. 

## Results and Discussion

### Random

Below are the result of both `Strahd` and `Leotris` using actions at random.

![](results/random.png)

The above was used to evaluate whether goal 1 had been achieved.

### Dueling Double Deep Q-Learning Network

Using a dueling double deep Q network, the follow results were achieved:

![dddqn_results](results/double_dueling_DQN.png)

The results above shows a failure to learn a reasonable strategy. Similar results were observed with vanilla DQN and double DQN. 

I believe the largest contributing factor is the use of a linear decay $\epsilon$-greedy exploration strategy. Specfically, the strategy employed to achieve the results above started with an $\epsilon$ exploration probability of 90%. That is to say, 10% of the time, the agent would take the action believed to be optimal, and the remaining 90% of the time, the agent would "explore" by taking a random action. The $\epsilon$ value would decay linearly down to a 5% probability of taking a random action. The agent seemed to learn that the action `ShootArrow` was the best action to perform regardless of the `current_state`. 

As a consequence, the agent would attempt to `ShootArrow` despite the fact that it had already used the action earlier within the same turn. As a result, the action would be ignored and the agent would be prompted for its next action until the `EndTurn` action was chosen or the time limit was reached. The agent never learned when to take the `EndTurn` action consistently resulted a `Timeout` terminal state.

Here are some additional hypothesis about why this problem was happening and what I tried to do to correct for it.

1. **$\epsilon$ decay rate was too fast**: Perhaps the neural network did not have enough time/epochs to learn that taking the `EndTurn` action would be beneficial. I decreased the decay rate by a factor of 100 but saw not significant changes. 
2. **Sparse rewards**: Perhaps only providing a reward for achieving a victory was too sparse. To address this, I structured the rewards such that the agent was rewarded every time it was able to do damage. This helped a great deal with the agent even learning to alter between `ShootArrow` and `EndTurn`. However, when the agent was left to run for a long enough time, eventually, the `TimeOut` terminal state was take over. 
![damage_reward_results](results/damage_reward_dddqn.png)

3. **Off-policy learning**: DQNs optimize for the case where the $\epsilon$-greedy strategy is replaced by one that is always taking the optimal action without exploration. This is a result of DQNs being off policy. Intuitively, I thought that this rectify the problem of `ShootArrow` being the only action chosen. This was because the off-policy learning would optimize for a case where the agent would not depend on `EndTurn` randomly being chosen. Despite this intuition, and full of desperation, I implemented SARSA as a quick test for on policy learning. Results were not improved. 
4. **Catastrophic forgetting**: Looking at [this](https://ai.stackexchange.com/questions/10822/what-is-happening-when-a-reinforcement-learning-agent-trains-itself-out-of-desir) stack exchange post, the user is asking why DQNs sometimes train themselves out of desired behavior. This was observed when the rewards were altered to reward any damage being delt. One answer given suggested that this could be a result of "catastrophic forgetting" (detailed in the post). To combat this, I tried to decrease the learning rate and use prioritized experience replay. Both of these did not work. Another suggestion was to keep experiences from early stages of exploration within memory. I have not tried this one yet. 



### Proximal Policy Iteration (PPO)

A simple implementation of [PPO](https://arxiv.org/abs/1707.06347) managed to obtain the best results:

![](results/PPO.png)

I believe that there were a couple contributing factors in obtaining these results. First, the learning rate had to be sufficiently small (alpha = 1e-5). With a larger learning rate of (alpha 1e-3), the following results were observed:

![](results/PPO_high_alpha.png)

In the case depicted above, the agent seems to have gotten into a parameter space in which it could not recover from

Second, the agent did not use an epsilon greedy like exploration strategy. Intead, PPO selects actions in a more stochastic nature compared to the epsilon greedy approach once its reached low exploration states. As a result, this avoided the problem observed when using variants of Q-learning.

## Conclusion